In [ ]:
import os
from dotenv import load_dotenv
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_groq import ChatGroq

# Load environment variables
load_dotenv()
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
if not GROQ_API_KEY:
    raise ValueError("Missing GROQ_API_KEY in .env file")

# Figure out root path (works both from notebooks/ or project root)
ROOT_DIR = os.path.abspath(os.path.join(os.getcwd(), "..")) \
    if os.path.basename(os.getcwd()) == "notebooks" else os.getcwd()

CHROMA_DIR = os.path.join(ROOT_DIR, "chroma_db")
EMBED_MODEL = "BAAI/bge-large-en-v1.5"
MODEL_NAME = "llama-3.3-70b-versatile"

print(f"Using Chroma DB from: {CHROMA_DIR}")

# Load saved embeddings and vector store
embeddings = HuggingFaceEmbeddings(model_name=EMBED_MODEL)
db = Chroma(persist_directory=CHROMA_DIR, embedding_function=embeddings)
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 5})

# Initialize Llama 3.3 (GroqCloud)
llm = ChatGroq(
    api_key=GROQ_API_KEY,
    model=MODEL_NAME,
    temperature=0.2,
    max_tokens=512,
)

# Prompt format
prompt = PromptTemplate(
    template="""
You are a helpful legal assistant. Use the context below to answer clearly and accurately.

Context:
{context}

Question:
{question}

Answer:
""",
    input_variables=["context", "question"],
)

# Create the retrieval-based QA chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff",
    chain_type_kwargs={"prompt": prompt},
    return_source_documents=True,
)


In [ ]:
# Try a sample query
query = "Is it illegal to use bombs in nepal?"
result = qa_chain({"query": query})

print("\nAnswer:")
print(result["result"])

print("\nSources:")
for doc in result["source_documents"]:
    print(f"- {doc.metadata.get('source')} (page {doc.metadata.get('page')})")


Answer:
Yes, it is illegal to use bombs in Nepal. According to the context provided, specifically Section 138, the production, making, or preparation of explosives, including bombs, without a license or in a manner contrary to the terms and restrictions specified in the license, is prohibited. Additionally, the explanation under Section 138 defines "explosives" to include bombs, and "bomb" is defined as a device made to self-detonate or detonate with the help of other substances. Therefore, using bombs in Nepal is considered an offense related to explosives and is subject to legal penalties.

Sources:
- criminal-code-nepal.pdf (page 86)
- criminal-code-nepal.pdf (page 40)
- criminal-code-nepal.pdf (page 38)
- criminal-code-nepal.pdf (page 39)
- criminal-code-nepal.pdf (page 32)


In [3]:
import os
from dotenv import load_dotenv
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_groq import ChatGroq

# Load environment variables
load_dotenv()
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
if not GROQ_API_KEY:
    raise ValueError("Missing GROQ_API_KEY in .env file")

In [4]:
# Figure out root path (works both from notebooks/ or project root)
ROOT_DIR = os.path.abspath(os.path.join(os.getcwd(), "..")) \
    if os.path.basename(os.getcwd()) == "notebooks" else os.getcwd()

CHROMA_DIR = os.path.join(ROOT_DIR, "chroma_db")
EMBED_MODEL = "BAAI/bge-large-en-v1.5"
MODEL_NAME = "llama-3.3-70b-versatile"

print(f"Using Chroma DB from: {CHROMA_DIR}")

Using Chroma DB from: /home/jagannath/myPocketLawyer/chroma_db


In [5]:
# Load saved embeddings and vector store
embeddings = HuggingFaceEmbeddings(model_name=EMBED_MODEL)
db = Chroma(persist_directory=CHROMA_DIR, embedding_function=embeddings)
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 5})

# Initialize Llama 3.3 (GroqCloud)
llm = ChatGroq(
    api_key=GROQ_API_KEY,
    model=MODEL_NAME,
    temperature=0.2,
    max_tokens=512,
)

# Prompt format
prompt = PromptTemplate(
    template="""
You are a helpful legal assistant. Use the context below to answer clearly and accurately.

Context:
{context}

Question:
{question}

Answer:
""",
    input_variables=["context", "question"],
)

/tmp/ipykernel_43106/3278185895.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=EMBED_MODEL)
/tmp/ipykernel_43106/3278185895.py:3: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  db = Chroma(persist_directory=CHROMA_DIR, embedding_function=embeddings)


In [6]:
# Create the retrieval-based QA chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff",
    chain_type_kwargs={"prompt": prompt},
    return_source_documents=True,
)

In [9]:
# Try a sample query
query = "what measure should i apply first if someone sue me?"
result = qa_chain({"query": query})

print("\nAnswer:")
print(result["result"])

print("\nSources:")
for doc in result["source_documents"]:
    print(f"- {doc.metadata.get('source')} (page {doc.metadata.get('page')})")


Answer:
If someone sues you, the first measure you should apply is to seek legal counsel and understand the nature of the claim against you. However, based on the provided context, it appears that the laws mentioned are related to compensation and restitution in criminal cases.

In this context, if you are being sued for compensation, you should first try to negotiate a settlement or payment plan with the plaintiff. If this is not possible, you may want to consider furnishing property as security for the compensation, as mentioned in Section 42(2), which allows the court to order payment in installments over a maximum period of one year.

It's essential to note that the specific laws and procedures may vary depending on the jurisdiction and the nature of the claim. Therefore, it's crucial to consult with a legal professional to determine the best course of action in your specific situation.

Sources:
- nepal-act-for-determination-and-execution-of-sentences-for-criminal-offences-englis